# **Previsão de Pouso do Primeiro Estágio de Foguetes Falcon 9**


### Data wrangling


Nesta seção, será realizado uma Análise Exploratória de Dados (AED) para identificar padrões nos dados e determinar qual será o rotúlo para treinar os modelos de previsão supervisionados

No conjunto de dados, existem vários casos diferentes em que o booster não pousou com sucesso. Isso se dá, devido à algum tipo de acidente que ocorre durante o processo de pouso. A partir do dataframe que foi criado na seção anterior, temos que, na coluna 'Outcome' (resultado do pouso), <code>True Ocean</code> significa que o pouso foi bem sucedido em uma determinada região do oceano, enquanto <code>False Ocean</code> significa que o pouso não foi bem sucedido em uma determinada região do oceano. <code>True RTLS</code> significa que o pouso foi bem sucedido em um bloco terrestre, já <code>False RTLS</code> significa que o pouso não foi bem sucedido em um bloco terrestre. <code>True ASDS</code> significa que o pouso foi bem sucedido em um navio drone, enquanto <code>False ASDS</code> significa que o pouso não foi bem sucedido em um navio drone.

Diante disso, nesta seção iremos focar em converter os resultados dos pousos anteriormente expostos em rótulos para treinamento dos modelos de previsão, sendo `1` para pousos bem sucedidos e `0` para pousos que não foram bem sucedidos.

### Objetivos

*   Realizar uma Análise Exploratória dos Dados (AED);
*   Determinar rótulos para treinar os modelos de previsão.


***


### Importação das bibliotecas

In [1]:
# Pandas: biblioteca utilizada para manipulação e análise de dados
import pandas as pd
# NumPy: biblioteca utilizada trabalhar com matrizes além de possibilitar o uso de funções matemáticas de alto nível para operar matrizes
import numpy as np

### Análise dos dados


Carregamento do conjunto de dados criado na seção anterior:

In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identificação e cálculo da porcentagem de valores ausentes em cada atributo exposto na tabela:

In [3]:
df.isnull().sum()/df.count()*100

FlightNumber       0.000
Date               0.000
BoosterVersion     0.000
PayloadMass        0.000
Orbit              0.000
LaunchSite         0.000
Outcome            0.000
Flights            0.000
GridFins           0.000
Reused             0.000
Legs               0.000
LandingPad        40.625
Block              0.000
ReusedCount        0.000
Serial             0.000
Longitude          0.000
Latitude           0.000
dtype: float64

Identificação quais colunas possuem valores numéricos e categóricos

In [4]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### Calculo do número de lançamentos realizados em cada local

Os dados da SpaceX apresentam as diversas instalações de lançamento de foguetes: https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01

Cape Canaveral Space Launch Complex 40: <b>CCAFS SLC 40</b>;

Vandenberg Air Force Base Space Launch Complex 4E: <b>VAFB SLC 4E </b>;

Kennedy Space Center Launch Complex 39A: <b>KSC LC 39A </b>;

A localização de cada uma das instalações listadas encontram-se na coluna <code>LaunchSite</code>


Verificação do número de lançamentos realizados em cada uma das instalações anteriormente expostas:

In [5]:
# Usa o método value_counts() na coluna 'LaunchSite' para contar o número de lançamentos em cada instalação
df['LaunchSite'].value_counts()

CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: LaunchSite, dtype: int64

Cada lançamento visa uma órbita específica. A seguir seram apresentadas as órbitas de interesse para o nosso estudo:

*   <b> Geostationary Orbit (GEO)</b> A Órbita Geoestacionária é uma órbita geossíncrona circular, localizada à 35,786 quilômetros acima da linha do equador e segue a mesma direção da rotação da Terra. <a href="https://en.wikipedia.org/wiki/Geostationary_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01"> \[1] </a>

*   <b> Medium Earth Orbit (MEO) </b> A Órbita Terrestre Média trata-se de uma órbita geocêntrica que varia em altitude de 2.000 km até logo abaixo da órbita geossíncrona a 35.786 quilômetros, possuindo um período orbital de 12 horas. <a href="https://en.wikipedia.org/wiki/List_of_orbits?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01"> \[2] </a>

*   <b>Highly Elliptical Orbit (HEO)</b> A Órbita Altamente Elíptica, é uma órbita elíptica com alta excentricidade, geralmente referindo-se a uma em torno da Terra. <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">\[3]</a>.

*   <b>Low Earth Orbit (LEO)</b>: A Órbita Terrestre Baixa é uma órbita centrada na Terra com uma altitude de 2.000 km ou menos. A maioria dos objetos feitos pelo homem no espaço sideral estão em LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01'>\[4]</a>.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### Calculo do número e a ocorrência de cada orbita


Utiliza o método <code>.value_counts()</code> para determinar o número e a ocorrência de cada órbita na coluna '<code>Orbit</code>'


In [6]:
df['Orbit'].value_counts()

GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: Orbit, dtype: int64

### Cálculo do número e ocorrência do resultado da missão de acordo com a órbita

Utiliza o método <code>.value_counts()</code> na coluna '<code>Outcome</code>' para determinar o número de '<code>landing_outcomes</code>' e depois atribui o resultado final a uma variável 'landing_outcomes'.


In [23]:
# landing_outcomes = valores na coluna 'Outcome'
landing_outcomes = df['Outcome'].value_counts()
landing_outcomes

True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
None ASDS       2
False Ocean     2
False RTLS      1
Name: Outcome, dtype: int64

Significado dos valores da variável 'landing_outcomes':

* <code>True Ocean</code>: o pouso foi bem sucedido em uma determinada região do oceano;

* <code>False Ocean</code>: o pouso falhou em uma determinada região do oceano;

* <code>True RTLS</code>: o pouso foi bem sucedido em uma plataforma terrestre;

* <code>False RTLS</code>: o pouso falhou em uma plataforma terrestre;

* <code>True ASDS</code>: o pouso foi bem sucedido em um navio drone;

* <code>False ASDS</code>: o pouso falhou em um navio drone;

* <code>None ASDS</code> e <code>None None</code>: representa as falhas nos pousos.

Criação de conjunto de dados contendo apenas os resultados de pouso falhos:


In [10]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 None ASDS
6 False Ocean
7 False RTLS


In [11]:
# Cria um conjunto de dados contendo apenas os valores dos pousos falhos
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### Criação de rótulos para a variável 'landing_outcome'


In [16]:
# landing_class = 0 para pousos falhos 'bad_outcome'
# landing_class = 1 caso contrário
landing_class = []
for i in df['Outcome']:
    if i in bad_outcomes:
        landing_class.append(0)
    else:
        landing_class.append(1)


Essa variável representará a variável de classificação que apresenta o resultado de cada lançamento. Se o valor for <code>'0'</code>, o primeiro estágio falhou ao tentar pousar; se o valor for <code>'1'</code> então o pouso do primeiro estágio foi bem sucedido.

In [17]:
# Cria uma coluna 'class' contendo as classificações dos pousos do primeiro estágio e insere no dataframe 'df'
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [18]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


Para cálculo da taxa de sucesso nas tentativas de pouso:

In [19]:
df["Class"].mean()

0.6666666666666666

Exportação do dataframe final para o formato CSV


In [21]:
df.to_csv("dataset_part_2.csv", index=False)